In [1]:
from utils.data_preprocess import *
import pandas as pd
import numpy as np
import tensorflow as tf
import talib as ta

In [2]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')

In [17]:
def func(df):
    if (df['Trend'] == 0) or (df['Trend'] == 1 and df['Previous_Trend'] == 1) or (df['Trend'] == -1 and df['Previous_Trend'] == -1):
        return 'Hold'
    elif (df['Trend'] == 1) and (df['Previous_Trend'] == 0 or -1):
        return 'Buy'
    elif (df['Trend'] == -1) and (df['Previous_Trend'] == 0 or 1):
        return 'Sell'

In [ ]:
# DataLabeling
data = df.set_index('Timestamp').iloc[:, :5]
data['STD'] = data.Close.rolling(24).std()
data['Next_Close'] = data.Close.shift(-24)
data = data.dropna()
data['Trend'] = np.where(data.Next_Close >= data.Close*(1+0.55*data.STD), 1,
                         np.where(data.Next_Close <= data.Close*(1-0.55*data.STD), -1, 0))
data['Previous_Trend'] = data.Trend.shift(fill_value=0)
data = data.drop(['STD'], axis=1)
# data['Next_Trend'] = data.Next_Trend.astype(int)
data = data.assign(Label=data.apply(func, axis=1))
data.head()

In [3]:
raw_data = DataLabeling(df, 24)

In [4]:
raw_data.labelled_data

c:\Users\dddru\Documents\GitHub\Is-Technical-Indicator-Really-a-Thing-for-Deep-Learning\utils\data_preprocess.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  np.where(data.Next_Close <= data.Close*(1-self.__alpha*data.STD), -1, 0))
c:\Users\dddru\Documents\GitHub\Is-Technical-Indicator-Really-a-Thing-for-Deep-Learning\utils\data_preprocess.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Previous_Trend'] = data.Trend.shift(fill_value=0)


,Open,High,Low,Close,Volume,Label
Timestamp,,,,,,
2017-07-14 05:55:00,0.085506,0.085782,0.085341,0.085399,13.8400,Buy
2017-07-14 06:00:00,0.085399,0.085460,0.085398,0.085460,0.6060,Hold
2017-07-14 06:05:00,0.085470,0.086204,0.085402,0.086186,101.7220,Hold
2017-07-14 06:10:00,0.086274,0.087001,0.086274,0.086973,59.8070,Hold
2017-07-14 06:15:00,0.087097,0.087097,0.086898,0.086899,28.8000,Sell
...,...,...,...,...,...,...
2021-11-10 10:55:00,0.070991,0.070999,0.070942,0.070982,85.0707,Hold
2021-11-10 11:00:00,0.070979,0.070979,0.070910,0.070947,82.8314,Hold
2021-11-10 11:05:00,0.070947,0.070967,0.070890,0.070934,147.1850,Buy
